In [24]:
import numpy as np
import wave
import pyaudio
import tensorflow as tf
import keras
import time

In [13]:
def rec_audio(filename):
    rec_time = 3            # 録音時間[s]

    file_path = filename 
    fmt = pyaudio.paInt16  # 音声のフォーマット
    ch = 1              # チャンネル1(モノラル)
    sampling_rate = 44100 # サンプリング周波数
    chunk = 2**11       # チャンク（データ点数）
    audio = pyaudio.PyAudio()
    index = 1 # 録音デバイスのインデックス番号（デフォルト1）

    stream = audio.open(format=fmt, channels=ch, rate=sampling_rate, input=True,
                        input_device_index = index,
                        frames_per_buffer=chunk)
    # 録音処理
    frames = []
    for i in range(0, int(sampling_rate / chunk * rec_time)):
        data = stream.read(chunk)
        frames.append(data)

    # 録音終了処理
    stream.stop_stream()
    stream.close()
    audio.terminate()

    # 録音データをファイルに保存
    wav = wave.open(file_path, 'wb')
    wav.setnchannels(ch)
    wav.setsampwidth(audio.get_sample_size(fmt))
    wav.setframerate(sampling_rate)
    wav.writeframes(b''.join(frames))
    wav.close()

In [14]:
def wave_load(filename):
    # open wave file
    wf = wave.open(filename,'r')
    channels = wf.getnchannels()

    # load wave data
    chunk_size = wf.getnframes()
    amp  = (2**8) ** wf.getsampwidth() / 2
    data = wf.readframes(chunk_size)   # バイナリ読み込み
    data = np.frombuffer(data,'int16') # intに変換
    data = data / amp                  # 振幅正規化
    data = data[::channels]
   
    return data

In [17]:
def fft_load(size,filename):
    '''
    st = サンプリング開始位置 [s]
    size = FFTのサンプル数（２＊＊ｎ）
    filename = オーディオファイルのディレクトリ

    '''
    #st=44100*st
    
    
    hammingWindow = np.hamming(size)    # ハミング窓
    fs = 44100 #サンプリングレート
    d = 1.0 / fs #サンプリングレートの逆数
    freqList = np.fft.fftfreq(size, d)
    
    wave = wave_load(filename)
    #print(len(wave))
    
    #10000Hzごとにサンプルを10点取ってそれを平均したものを学習用の音声スペクトルデータとして用いる。
    for i in range(10):
        st=i*10000
        windowedData = hammingWindow * wave[st:st+size]  # 切り出した波形データ（窓関数あり）
        sample_data = np.fft.fft(windowedData)
        sample_data = sample_data / max(abs(sample_data)) # 0~1正規化
        if(i==0):
            data = sample_data
        else:
            data = (i*data+sample_data)/(i+1)
    
    data=abs(data)/max(abs(data))
    #print(type(data[0]))

    return data

### modelの読み出し 

In [11]:
model=keras.models.load_model("./model")

# リアルタイムで温度測定をしてみる。
---
インターバルを挟み、一旦音声ファイルを録音した上でスペクトルを学習にかける。  
出来れば音声保存を介さずに処理したい。

In [33]:
filename="./t_fal_audio/sound_data.wav"
size=2**12

interval=2 #何秒間インターバルを挟むか？ 1回：録音時間3s＋インターバル

temp_set=np.array([])
time_set=np.array([])

start_time=time.time()
lap_time=0

i=0
while(300-lap_time>0):
    if(i==0):print("処理開始")
    
    time.sleep(interval)
    
    rec_audio(filename)
    data=fft_load(size, filename)
    data=data.reshape(1,4096)
    
    temp=model.predict(data)*100
    lap_time=time.time()-start_time
    
    temp_set=np.append(temp_set, temp)
    time_set=np.append(time_set, lap_time)
    
    i+=1
    print("{num}回目の処理が終了".format(num=i))

print("\n")

for i in range(len(time_set)):
    print("{0:.2f},{1:.2f}".format(time_set[i], temp_set[i]))

処理開始
1回目の処理が終了
2回目の処理が終了
3回目の処理が終了
4回目の処理が終了
5回目の処理が終了
6回目の処理が終了
7回目の処理が終了
8回目の処理が終了
9回目の処理が終了
10回目の処理が終了
11回目の処理が終了
12回目の処理が終了
13回目の処理が終了
14回目の処理が終了
15回目の処理が終了
16回目の処理が終了
17回目の処理が終了
18回目の処理が終了
19回目の処理が終了
20回目の処理が終了
21回目の処理が終了
22回目の処理が終了
23回目の処理が終了
24回目の処理が終了
25回目の処理が終了
26回目の処理が終了
27回目の処理が終了
28回目の処理が終了
29回目の処理が終了
30回目の処理が終了
31回目の処理が終了
32回目の処理が終了
33回目の処理が終了
34回目の処理が終了
35回目の処理が終了
36回目の処理が終了
37回目の処理が終了
38回目の処理が終了
39回目の処理が終了
40回目の処理が終了
41回目の処理が終了
42回目の処理が終了
43回目の処理が終了
44回目の処理が終了
45回目の処理が終了
46回目の処理が終了
47回目の処理が終了
48回目の処理が終了
49回目の処理が終了
50回目の処理が終了
51回目の処理が終了
52回目の処理が終了
53回目の処理が終了
54回目の処理が終了
55回目の処理が終了
56回目の処理が終了
57回目の処理が終了
58回目の処理が終了


5.18,29.66
10.35,28.96
15.53,26.45
20.72,31.87
25.90,37.33
31.10,31.00
36.27,31.57
41.45,33.29
46.62,35.32
51.80,36.17
56.98,36.06
62.15,43.76
67.33,34.06
72.51,45.51
77.69,54.29
82.86,51.73
88.04,42.56
93.22,55.64
98.40,59.43
103.58,51.61
108.76,62.59
113.93,46.95
119.11,44.60
124.28,57.50
129.46,58.38
134.64,59.50
139.81,49.78
144.99,52.51
150.17,61.65
155.34,